<center> <img style="float: center;" src="images/CI_horizontal.png" width="450">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span> 

# <center>Inference: Imputing Missing Values in a Dataset</center>
<br>
<center>Rukhshan Mian, Maryah Garner, Brian Kim</center>
<br>
<a href="https://doi.org/10.5281/zenodo.6463899"><img src="https://zenodo.org/badge/DOI/10.5281/zenodo.6463899.svg" alt="DOI"></a>


## Introduction: Missing Data 

Datasets often contain variables with missing (or unknown) data. This notebook explores methods for treating missing values in a dataset. Unfortunately, there is usually no single *right* answer for how to treat missing values. But it is possible to impute missing values to infer--or provide a best guess for--each missing point's true value. 



### Learning Objectives

#### Explore common methods for dealing with missing values:
- [Dropping all "missing" values](#dropping-values)
    
**Imputation methods**
- [Substituting missing values with the mean of a group](#mean-imputation) 
- [Regression imputation](#regression-imputation)
- [Mode imputation (for categorical variables)](#mode-imputation)


<span style="color:blue">**The notebook will walk through imputing missing values for age at dissertation for the 2015 cohort.**</span>
***

## R Setup

As always, start by importing the required libraries, as well as creating the connection to the database.

In [ ]:
options(warn=-1)                     # turn off warnings

suppressMessages(library(odbc))      #database interaction imports

suppressMessages(library(tidyverse)) # for data manipulation/visualization

suppressMessages(library(scales))    # scaling data, calculating percentages, overriding default graphing
 
suppressMessages(library(survey))    # adding weights

# to better view images
# For easier viewing of graphs
# Adjust repr.plot.width and repr.plot.height to change the size of graphs
theme_set(theme_gray(base_size = 24))
options(repr.plot.width = 20, repr.plot.height = 12)
options(scipen=999)
options(warn=0)

In [ ]:
# Connect to the database
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

## Read in data

In [ ]:
# See how many missing ages there are
# Get the count of non-null values for age at dissertation and the count of all rows
query <- "
SELECT count(age_at_dissertation) AS age_at_dissertation_count, 
    count(*) AS total_count, 
    count(*)-count(age_at_dissertation) AS total_missing
FROM ds_nsf_ncses.dbo.nsf_sed;
"
dbGetQuery(con,query)

In [ ]:
# First, confirm the data type of the age_at_dissertation variable
query <- "
SELECT age_at_dissertation, drf_id, phdinst
FROM ds_nsf_ncses.dbo.nsf_sed;
"
df <- dbGetQuery(con, query)

class(df$age_at_dissertation)

### Checking for Missing Data 

Our example focuses on the `age_at_dissertation` variable in the `nsf_sed` table, where there are a lot of missing values for age.

> Given that there are a sizable amount of observations with missing ages, imputation might be suitable if you wanted to include age at dissertation in your analysis. The following subsections will walk through different methods of imputation.

<a id="dropping-values"></a>
# 
### Method 1. Dropping missing values

Oftentimes, missing values are treated by ignoring them (dropping them). However, ignoring missing values assumes that they represent the same distribution as the present one. 
> Dropping missing values is often called listwise deletion and essentially assumes that missing values are missing completely at random (MCAR), which is not always the case.

**We recommend you do not ignore missing values.** However, if you are going to drop missing values, dig deeper to understand who is part of the cohort being dropped. For example: *What is the gender/racial composition of the cohort that is being dropped? Are there specific genders/races that are being dropped?*

<br>

Before dropping missing values, we create labeled variables for **srceprim** (primary source of support), **sex**, and **race** (based on the data dictionary for SED) to perform additional data exploration and see if there are certain types of doctorate students who are being dropped.

In [ ]:
# Let's get the SED table
query <- "
SELECT drf_id, age_at_dissertation, phdfield_name, srceprim, sex, race
FROM ds_nsf_ncses.dbo.nsf_sed
"
sed <- dbGetQuery(con,query)

# In order to use sex and race, we would need to create labeled variables. 

sed <- sed %>%
    #creating source_cat – this is similar to what we did in Module 2
    mutate(source_cat = case_when(srceprim %in% c("A", "B") ~ "Fellowship Grant",
                                  srceprim == "C" ~ "Teaching Assistantship", 
                                  srceprim == "D" ~ "Research Assistantship",
                                  srceprim %in% c("H", "I", "J") ~ "Own Funds/Loan", 
                                  srceprim %in% c("E", "F", "G", "K", "L", "M","N") ~ "Other",
                                  srceprim == '' ~ 'Unknown')
        ) %>%
    #creating sex_cat: we are creating these categories based off the Data Dictionary
    mutate(sex_cat = case_when(sex == "" ~ "Unknown",
                            sex == "1" ~ "Male", 
                            sex == "2" ~ "Female")) %>%

    #creating race_cat: we are creating these categories based off the Data Dictionary
    mutate(race_cat = case_when(race == "NA" ~ "Unknown",
                               race == "-1" ~ "Refused", 
                               race == "1" ~ "American Indian or Alaskan Native", 
                               race == "2" ~ "Asian/Pacific Islander", 
                               race == "3" ~ "Asian", 
                               race == "4" ~ "Native Hawaiian or other Pacific Islander", 
                               race == "5" ~ "Black or African American", 
                               race == "6" ~ "Puerto Rican", 
                               race == "7" ~ "Mexican or Chicano", 
                               race == "8" ~ "Cuban",
                               race == "9" ~ "Other Hispanic - Specify", 
                               race == "10" ~ "White",
                               race == "11" ~ "Multiple racial responses cannot be prioritized into another category",
                               race == "12" ~ "Other"))

# drop the original sex, race and srceprim columns 
# we will not be needing these as we just created labelled versions (source_cat, sex_cat, race_cat)
sed <- sed %>% 
    select(-c(sex, race, srceprim))

head(sed)

#### Who is being dropped?

Here, we see if there are any sex, race, or PhD field categories that are being dropped more often than others when we drop missing values in **age_at_dissertation** to calculate the mean age.

##### **Sex**

In [ ]:
sed %>%
    # group by sex
    group_by(sex_cat) %>% 
    # count the number of unique ids
    mutate(complete_count = n_distinct(drf_id)) %>%
    # ungroup
    ungroup() %>%
    # keep non missing age_at_dissertation
    filter(!is.na(age_at_dissertation)) %>% 
    # regroup by sex
    group_by(sex_cat) %>% 
    # get the counts for each case: 
    summarize(complete_count = mean(complete_count), # complete count
              count_after_dropping_missing = n_distinct(drf_id), # count after dropping missing ages
             count_with_just_missing = complete_count-count_after_dropping_missing) %>%  # count when we just keep missing ages
    ungroup() %>% # ungroup
    mutate(percentage_dropped = round((count_with_just_missing/complete_count)*100, 1)) # calculated rounded percentages


We observe how the most values being dropped are for the Unknown category.

##### **Race**

In [ ]:
sed %>%
    # group by race
    group_by(race_cat) %>% 
    # count the number of unique ids
    mutate(complete_count = n_distinct(drf_id)) %>%
    # ungroup
    ungroup() %>%
    # keep non missing age_at_dissertation
    filter(!is.na(age_at_dissertation)) %>% 
    # regroup by race
    group_by(race_cat) %>% 
    # get the counts for each case: 
    summarize(complete_count = mean(complete_count), # complete count
              count_after_dropping_missing = n_distinct(drf_id), # count after dropping missing ages
             count_with_just_missing = complete_count-count_after_dropping_missing) %>%  # count when we just keep missing ages
    ungroup() %>% # ungroup
    mutate(percentage_dropped = round((count_with_just_missing/complete_count)*100, 1)) # calculated rounded percentages


Here, we observe what happens to categories of race when we drop missing values in **age_at_dissertation**. We see that the Unknown and Refused categories are primarily dropped.

##### **PhD Field of Study**

In [ ]:
sed %>%    
    # group by phdfield name
    group_by(phdfield_name) %>% 
    # count the number of unique ids
    mutate(complete_count = n_distinct(drf_id)) %>%
    # ungroup
    ungroup() %>%
    # keep non missing age_at_dissertation
    filter(!is.na(age_at_dissertation)) %>% 
    # regroup by phd field name
    group_by(phdfield_name) %>% 
    # get the counts for each case: 
    summarize(complete_count = mean(complete_count), # complete count
              count_after_dropping_missing = n_distinct(drf_id), # count after dropping missing ages
             count_with_just_missing = complete_count-count_after_dropping_missing) %>%  # count when we just keep missing ages
    ungroup() %>% # ungroup
    mutate(percentage_dropped = round((count_with_just_missing/complete_count)*100, 1)) # calculated rounded percentages

In this case, we see that the category for Unknown Field is primarily dropped. The goal of this exercise is not to see why certain categories are being dropped but just to better understand the data we are using.

<a id="mean-imputation"></a>
## Method 2. Mean Imputation

Imputed values will be **approximations**, and must be treated as such. If choosing to impute missing values in your project or future work, acknowledge the process you use and clearly state for which variables the values are imputed.

One of the simplest ways of imputing values is by taking the mean and filling it in for the missing values. It's possible to do this by using the overall mean, as well as by certain subgroups. 



#### Calculating Mean

In [ ]:
# Find the overall mean by excluding missing values
overall_mean = mean(sed$age_at_dissertation, na.rm=TRUE)

overall_mean

In [ ]:
# set missing means to the overall mean
complete_sed <- sed %>%
    # using ifelse to replace missing ages with mean age
    mutate(age_at_dissertation = ifelse(is.na(age_at_dissertation),  # missing age
                  overall_mean,                # value to replace missing age with 
                  age_at_dissertation))              # value if age is not missing

In [ ]:
head(complete_sed)

# 
# 

Imputation can also be used with more granular **subgroup means**, like within each combination of **phdfield_name**&mdash;**race_cat**. This assumes that the missing **age_at_dissertation** values are conditional on PhD field name and race.

In [ ]:
# calculate non-missing ages within each phdfield_name, sex and race
mean_by_group <- sed %>%
                # grouping by phd field and race
                group_by(phdfield_name, race_cat) %>%
                # calculating mean by phd field and race after removing missing values
                summarise(mean_by_group = mean(age_at_dissertation, na.rm=TRUE))

head(mean_by_group)

Next, we find the number of combinations for whom we have a missing mean.

In [ ]:
# checking to see missing values for mean_by_group
table(is.na(mean_by_group$mean_by_group))

In [ ]:
mean_by_group %>%
    filter(is.na(mean_by_group))

We can not impute the age at disertation for SED respondance who have unknown values for both phdfield_name and race_cat. 


After calculating the mean non-missing ages within each PhD field and race, these means can be joined to the original `sed` data frame.

In [ ]:
# join mean_by_group with sed by phdfield_name
sed_phd <- sed %>%
    # performing an inner join based on phd field and race
    inner_join(mean_by_group, by = c('phdfield_name', 'race_cat'))

head(sed_phd)

The imputed ages can now be calculated where missing **age_at_dissertation** values will take on the appropriate **mean_by_group** calculation and non-missing values will remain the same.

In [ ]:
# create imputed age column
sed_phd <- sed_phd %>%
    mutate(
        # create imp age
        imp_age = ifelse( # using ifelse
            is.na(age_at_dissertation), # if age_at_dissertation is missing
            mean_by_group, # replace imp_age with mean_by_group (mean by phd field & race combination)
            age_at_dissertation # else replace imp_age with non-missing age_at_dissertation value
        )
    )

head(sed_phd)

In [ ]:
# see means after imputation
mean(sed_phd$imp_age, na.rm=TRUE)

<a id="regression-imputation"></a>
### Method 3. Regression Imputation

Regression can also be used to try to compute more accurate values. A regression equation will be built with the observations for which the age is known, and then applied to essentially predict the missing values. This is, in effect, an extension of the mean imputation by subgroup. Here, the **primary source of support** as well as the **PhD field** and **race** variables will be used as predictors to generate the regression.

> Note: Here the assumptions associated with linear regressions are not checked, as this example is aimed at merely displaying how to use a linear regression for imputation. If planning on using regression imputation, please check all assumptions before employing a predictive model.

Since the model will be built based on information only from the members of the cohort with non-missing age values, the `sed` dataset can be split into two datasets: one for training (`sed_nona`), without any missing data for the training of the model&mdash;and one for testing (`sed_na`), with only missing data to use for prediction.

In [ ]:
# Create a dataset with no missing data, for the training of the model
sed_nona <- sed %>%
            drop_na()

In [ ]:
# Create a dataset with only missing data, to use for prediction
# Remove a column with age as it is always null
sed_na <- sed %>%
    filter(is.na(age_at_dissertation)) %>%
    select(-c(age_at_dissertation))

# 
The model creation process for a linear regression can be done using the `lm()` function. The variable to predict is on the left-hand side of `lm()` before the `~`, and the predictors (**phdfield_name**, **source_cat**, **race_cat** in this case) are on the right-hand side of the `~`.

In [ ]:
# run model and fit coefficients
model <- lm(age_at_dissertation ~ phdfield_name + source_cat + race_cat, data = sed_nona)

Now that the model has been fit, the missing ages can be predicted using the coefficients.

In [ ]:
# predict age for test set
pred_age <- data.frame(age_at_dissertation = predict(model, newdata=sed_na))

head(pred_age)

Because the output for `predict()` retains the same order of rows from `sed`, the **age_at_dissertation** variable from **pred_age** can be easily added into the existing `sed` data frame.

In [ ]:
# save updated data frame
sed_na_w_age <- cbind(sed_na, pred_age)

# see the first 6 rows of the updated data frame
head(sed_na_w_age)

# 
Finally, before seeing the effects of the imputation method, the training set should be combined to get the full set of **age_at_dissertation** values, both imputed and previously known ones.

In [ ]:
# save combined training and testing sets
sed_all <- rbind(sed_na_w_age, sed_nona)

head(sed_all)

The entire age distribution can now be viewed.

In [ ]:
# see age distribution for full cohort
summary(sed_all$age_at_dissertation)

In [ ]:
# see age distribution for imputed portion of cohort
summary(sed_na_w_age$age_at_dissertation)

<a id="mode-imputation"></a>
# 
## Method 4. Mode Imputation (for Categorical Variables)

For categorical variables, it is impossible to apply mean imputation because there is not a mean to calculate. A method for imputing missing categorical values is to find the most frequent value (mode), and impute using the mode.

This example will cover using mode imputation for missing values of primary source of funding (**source_cat**) based on each **phdfield_name**&mdash;**race_cat** combination.

First, as exhibited in previous methods, remove the observations with missing values. Then, find the most frequent race per field of study. 

In [ ]:
# create a dataframe with only non-missing source of support
no_miss_prim <- sed %>% 
    filter(source_cat != 'Unknown')

# 
# 
The `group_by` function, in combination with `top_n`, can be used together to isolate the most common **source of support** value within each **phdfield_name**&mdash;**race_cat** combination.

In [ ]:
# Get only the most frequent race values per phdfield-race group
prim_mode <- no_miss_prim %>%
                # grouping by phd field and race
                group_by(phdfield_name, race_cat) %>%
                # counting each source of support within each phd field and race combination
                count(source_cat) %>%
                # keeping the source of support with the highest count
                top_n(1) %>%
                # removing the count variable (n -> created when we run count(source_cat) above)
                select(-c(n))

`prim_mode` can now function as a lookup table with the most frequent value of source of support per PhD field and race combination. This lookup table can now be joined with the original `sed` table. This will create two columns: **source_cat.x** (original value for source of support) and **source_cat.y** (source of support value from the lookup table).

In [ ]:
# merge original and lookup tables
merged <- left_join(sed, prim_mode, by=c('phdfield_name', 'race_cat'))

>The NA values in the **source_cat.x** column can be replaced with the known values in the column **source_cat.y**, and then these two columns (**source_cat.y** and **source_cat.x**) can be removed, leaving a new column "**source_imp**" with the imputed source of funding.

In [ ]:
# Replace the NA values in the `source_cat.x` column with the known values in the column `source_cat.y`
merged <- merged %>%
    mutate(
        # using ifelse to replace missing sources of support with imputed sources of support
        source_imp = ifelse(source_cat.x == "Unknown", source_cat.y, source_cat.x)
    ) %>%
    # deselecting unnecessary columns
    select(-c(source_cat.x, source_cat.y))

head(merged)

In [ ]:
# Examine the imputed mode for each category of funding
table(merged$source_imp)

In [ ]:
# Compare to the original category
table(sed$source_cat)

## Important

While performing imputations, there are a few things that we might need to look into further. If you observe that your results are changing dramatically, it is important to check if the imputation method you utilized is driving your results. If you are imputing missing values, it is recommended that you utilize several methods and compare the results.

Furthermore, one major challenge that you may face in your research when imputing values with a lot of missingness is that some groups do not get imputed at all. We encourage you to identify these groups (or categories) to better understand why we see missing values even after imputation.


## Closing the database connection

In [ ]:
# Close the database connection
dbDisconnect(con)